## 测试 rust 版本 czsc 库的使用

----

In [7]:
from rs_czsc import Event, Signal, CZSC

s1 = Signal(key="30分钟_D1_表里关系V230101", value="向上_任意_任意_0")
print(s1)

s2 = Signal(k1="60分钟", k2="D1", k3="表里关系V230101", v1="向上", v2="任意", v3="任意", score=0)
print(s2)

s3 = Signal("30分钟_D1_表里关系V230101_向上_任意_任意_0")
print(s3)

Signal('30分钟_D1_表里关系V230101_向上_任意_任意_0')
Signal('60分钟_D1_表里关系V230101_向上_任意_任意_0')
Signal('30分钟_D1_表里关系V230101_向上_任意_任意_0')


In [11]:
s1.to_string()

'30分钟_D1_表里关系V230101_向上_任意_任意_0'

### 测试 Event 对象
---

In [9]:
# python 版本：https://github.com/waditu/czsc/blob/0.9.68/czsc/objects.py#L630
from rs_czsc import Event, Signal, Operate


event = Event(name='平多', operate=Operate.LE, 
              signals_all=[
                        Signal("日线_D0停顿分型_BE辅助V230106_看空_强_任意_0"),
                ])

print(event.dump())

AttributeError: 'builtins.Event' object has no attribute 'dump'

In [1]:
from rs_czsc import Operate

In [ ]:
op = Operate.LE
assert op.value == "平多"

In [12]:
op.value

AttributeError: 'builtins.Operate' object has no attribute 'value'